# Evolver Loop 1 Analysis

Analyzing the baseline and identifying optimization opportunities.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import json

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

# Check N=1 in baseline
df = pd.read_csv('/home/submission/submission.csv')
n1_row = df[df['id'] == '001_0'].iloc[0]
print(f"N=1 baseline: x={n1_row['x']}, y={n1_row['y']}, deg={n1_row['deg']}")

# Parse the values
x = float(n1_row['x'][1:])  # Remove 's' prefix
y = float(n1_row['y'][1:])
angle = float(n1_row['deg'][1:])
print(f"Parsed: x={x}, y={y}, angle={angle}")

N=1 baseline: x=s-48.196086194214246, y=s58.770984615214225, deg=s45.0
Parsed: x=-48.196086194214246, y=58.770984615214225, angle=45.0


In [2]:
# For N=1, the position doesn't matter - only the rotation affects bounding box
# Let's find the optimal rotation for a single tree

def get_bbox_side(angle):
    """Get the bounding box side length for a tree at given angle."""
    tree = create_tree_polygon(0, 0, angle)
    bounds = tree.bounds  # (minx, miny, maxx, maxy)
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

# Current N=1 score
current_side = get_bbox_side(45.0)
current_score = current_side ** 2 / 1
print(f"Current N=1: angle=45°, side={current_side:.6f}, score={current_score:.6f}")

# Find optimal angle by exhaustive search
best_angle = 0
best_side = float('inf')
for angle_int in range(0, 36000):  # 0.01° increments
    angle = angle_int / 100
    side = get_bbox_side(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

optimal_score = best_side ** 2 / 1
print(f"\nOptimal N=1: angle={best_angle}°, side={best_side:.6f}, score={optimal_score:.6f}")
print(f"Improvement: {current_score - optimal_score:.6f} points")

Current N=1: angle=45°, side=0.813173, score=0.661250



Optimal N=1: angle=45.0°, side=0.813173, score=0.661250
Improvement: 0.000000 points


In [3]:
# Check the baseline per-N scores
with open('/home/code/experiments/000_baseline/metrics.json') as f:
    metrics = json.load(f)

per_n = metrics['per_n_scores']
print("Top 10 N values by score contribution:")
sorted_n = sorted(per_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_n[:10]:
    print(f"  N={n}: {score:.6f}")

print(f"\nTotal score: {sum(per_n.values()):.6f}")
print(f"Target: 68.888293")
print(f"Gap: {sum(per_n.values()) - 68.888293:.6f}")

Top 10 N values by score contribution:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.376949

Total score: 70.615791
Target: 68.888293
Gap: 1.727498


In [4]:
# Calculate theoretical minimum for N=1
# The tree has height 1.0 (from -0.2 to 0.8) and max width 0.7
# At 0° rotation, bbox is 0.7 x 1.0, so side = 1.0
# At 45° rotation, the diagonal increases
# At 90° rotation, bbox is 1.0 x 0.7, so side = 1.0
# The minimum side occurs at some angle where width ≈ height

print("Checking key angles:")
for angle in [0, 30, 45, 60, 90, 120, 135, 150, 180]:
    side = get_bbox_side(angle)
    print(f"  {angle}°: side={side:.6f}, score={side**2:.6f}")

print(f"\nOptimal angle: {best_angle}°")
print(f"Optimal side: {best_side:.6f}")
print(f"Optimal N=1 score: {best_side**2:.6f}")

Checking key angles:
  0°: side=1.000000, score=1.000000
  30°: side=0.903525, score=0.816358
  45°: side=0.813173, score=0.661250
  60°: side=0.903525, score=0.816358
  90°: side=1.000000, score=1.000000
  120°: side=0.903525, score=0.816358
  135°: side=0.813173, score=0.661250
  150°: side=0.903525, score=0.816358
  180°: side=1.000000, score=1.000000

Optimal angle: 45.0°
Optimal side: 0.813173
Optimal N=1 score: 0.661250


In [5]:
# Let's also check N=2 to N=5 to see if there's room for improvement
print("Analyzing N=2 to N=5:")
for n in range(2, 6):
    baseline_score = float(per_n[str(n)])
    # Theoretical minimum: if trees could be packed perfectly
    # For N trees, minimum area ≈ N * tree_area
    tree_area = Polygon(zip(TX, TY)).area
    min_area = n * tree_area
    min_side = np.sqrt(min_area)
    theoretical_min_score = min_side ** 2 / n
    print(f"  N={n}: baseline={baseline_score:.6f}, theoretical_min≈{theoretical_min_score:.6f}, gap={baseline_score - theoretical_min_score:.6f}")

Analyzing N=2 to N=5:
  N=2: baseline=0.450779, theoretical_min≈0.245625, gap=0.205154
  N=3: baseline=0.434745, theoretical_min≈0.245625, gap=0.189120
  N=4: baseline=0.416545, theoretical_min≈0.245625, gap=0.170920
  N=5: baseline=0.416850, theoretical_min≈0.245625, gap=0.171225


In [6]:
# Summary of optimization potential
print("=" * 50)
print("OPTIMIZATION POTENTIAL SUMMARY")
print("=" * 50)

# N=1 improvement
n1_improvement = current_score - optimal_score
print(f"\nN=1 improvement (exact): {n1_improvement:.6f} points")

# Estimate for N=2-10 (assume 10% improvement possible)
n2_10_total = sum(float(per_n[str(n)]) for n in range(2, 11))
print(f"N=2-10 total: {n2_10_total:.6f} points")
print(f"If 10% improvement: {n2_10_total * 0.1:.6f} points")

# Total potential
total_potential = n1_improvement + n2_10_total * 0.1
print(f"\nEstimated total potential from small N: {total_potential:.6f} points")
print(f"Current gap to target: {sum(per_n.values()) - 68.888293:.6f} points")
print(f"Remaining gap after small N optimization: {sum(per_n.values()) - 68.888293 - total_potential:.6f} points")

OPTIMIZATION POTENTIAL SUMMARY

N=1 improvement (exact): 0.000000 points
N=2-10 total: 3.667878 points
If 10% improvement: 0.366788 points

Estimated total potential from small N: 0.366788 points
Current gap to target: 1.727498 points
Remaining gap after small N optimization: 1.360711 points


# Investigating Overlap Issue in N=40

The submission failed with "Overlapping trees in group 040". Let's investigate.

In [2]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

# Check N=40 for overlaps
def parse_submission_full(df):
    """Parse submission dataframe into dict of n -> list of (x, y, angle)."""
    solutions = {}
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        x = float(row['x'][1:])  # Remove 's' prefix
        y = float(row['y'][1:])  # Remove 's' prefix
        angle = float(row['deg'][1:])  # Remove 's' prefix
        if n not in solutions:
            solutions[n] = []
        solutions[n].append((x, y, angle))
    return solutions

def check_overlaps(trees):
    """Check for overlapping trees."""
    polys = [create_tree_polygon(*t) for t in trees]
    overlaps = []
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]):
                intersection = polys[i].intersection(polys[j])
                if intersection.area > 1e-12:  # Non-trivial overlap
                    overlaps.append((i, j, intersection.area))
    return overlaps

# Load and check
df = pd.read_csv('/home/submission/submission.csv')
solutions = parse_submission_full(df)

print("Checking N=40 for overlaps...")
overlaps_40 = check_overlaps(solutions[40])
if overlaps_40:
    print(f"Found {len(overlaps_40)} overlapping pairs:")
    for i, j, area in overlaps_40[:10]:
        print(f"  Trees {i} and {j}: overlap area = {area:.2e}")
else:
    print("No overlaps detected with standard precision")

Checking N=40 for overlaps...
No overlaps detected with standard precision


In [3]:
# Check ALL N values for overlaps
print("Checking all N values for overlaps...")
overlapping_ns = []
for n in range(1, 201):
    overlaps = check_overlaps(solutions[n])
    if overlaps:
        overlapping_ns.append((n, len(overlaps), max(o[2] for o in overlaps)))
        
if overlapping_ns:
    print(f"\nFound overlaps in {len(overlapping_ns)} N values:")
    for n, count, max_area in overlapping_ns[:30]:
        print(f"  N={n}: {count} overlapping pairs, max area = {max_area:.2e}")
else:
    print("No overlaps detected with standard precision")

Checking all N values for overlaps...


No overlaps detected with standard precision


In [4]:
# Check sample submission for overlaps - it should be valid
sample_df = pd.read_csv('/home/data/sample_submission.csv')
sample_solutions = parse_submission_full(sample_df)

print("Checking sample submission for overlaps...")
sample_overlaps = []
for n in range(1, 201):
    overlaps = check_overlaps(sample_solutions[n])
    if overlaps:
        sample_overlaps.append((n, len(overlaps)))

if sample_overlaps:
    print(f"Sample has overlaps in {len(sample_overlaps)} N values")
else:
    print("Sample submission has NO overlaps - it's valid!")

Checking sample submission for overlaps...


Sample submission has NO overlaps - it's valid!


In [5]:
# Calculate sample submission score
def calculate_score(solutions):
    """Calculate total score for all N=1 to 200."""
    total = 0
    per_n_scores = {}
    for n in range(1, 201):
        if n not in solutions:
            continue
        trees = solutions[n]
        polys = [create_tree_polygon(*t) for t in trees]
        union = unary_union(polys)
        bounds = union.bounds
        side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
        contribution = (side ** 2) / n
        per_n_scores[n] = contribution
        total += contribution
    return total, per_n_scores

sample_score, sample_per_n = calculate_score(sample_solutions)
print(f"Sample submission score: {sample_score:.6f}")
print(f"Baseline score: 70.615791")
print(f"Difference: {sample_score - 70.615791:.6f}")

Sample submission score: 173.652299
Baseline score: 70.615791
Difference: 103.036508


In [ ]:
# The issue might be floating point precision
# Let's check the raw CSV values for N=40
print("Raw CSV values for N=40:")
n40_rows = df[df['id'].str.startswith('040_')]
print(n40_rows.to_string())

print("\n\nChecking precision of values:")
for _, row in n40_rows.iterrows():
    x_str = row['x'][1:]  # Remove 's'
    y_str = row['y'][1:]
    deg_str = row['deg'][1:]
    print(f"  {row['id']}: x has {len(x_str.split('.')[-1]) if '.' in x_str else 0} decimals")

In [ ]:
# Let's try a different approach: find a snapshot that Kaggle might accept
# by checking if any snapshot has simpler/rounder coordinates

import os
snapshot_base = '/home/nonroot/snapshots/santa-2025/'
snapshot_dirs = sorted(os.listdir(snapshot_base))

# Check the first few snapshots for their coordinate precision
print("Checking coordinate precision in different snapshots...")
for snap_dir in snapshot_dirs[:5]:
    sub_path = os.path.join(snapshot_base, snap_dir, 'submission', 'submission.csv')
    if os.path.exists(sub_path):
        df_snap = pd.read_csv(sub_path)
        # Check N=40 row 0
        n40_row = df_snap[df_snap['id'] == '040_0'].iloc[0]
        x_str = n40_row['x'][1:]
        print(f"{snap_dir}: x={x_str[:30]}...")

In [ ]:
# Strategy: Use the sample submission as a VALID baseline
# It scores 173.6 but at least it won't be rejected
# Then we can improve from there

# Or better: Create a hybrid - use sample for N=40 and baseline for others
# Let's check which N values in baseline might have precision issues

# First, let's understand what Kaggle's validation might be doing differently
# The issue could be:
# 1. Different floating point precision
# 2. Different polygon intersection algorithm
# 3. Rounding errors in coordinate parsing

# Let's create a "safe" submission by rounding coordinates
def round_coordinates(df, decimals=6):
    """Round coordinates to fewer decimal places."""
    df_new = df.copy()
    for col in ['x', 'y', 'deg']:
        df_new[col] = df_new[col].apply(lambda s: f"s{round(float(s[1:]), decimals)}")
    return df_new

# Round to 6 decimal places
df_rounded = round_coordinates(df, 6)
print("Rounded coordinates sample:")
print(df_rounded[df_rounded['id'].str.startswith('040_')].head())

In [ ]:
# Check if rounding causes overlaps
solutions_rounded = parse_submission_full(df_rounded)
print("Checking rounded submission for overlaps...")
rounded_overlaps = []
for n in range(1, 201):
    overlaps = check_overlaps(solutions_rounded[n])
    if overlaps:
        rounded_overlaps.append((n, len(overlaps)))

if rounded_overlaps:
    print(f"Rounded submission has overlaps in {len(rounded_overlaps)} N values:")
    for n, count in rounded_overlaps[:10]:
        print(f"  N={n}: {count} overlaps")
else:
    print("Rounded submission has NO overlaps!")
    
# Calculate score
rounded_score, _ = calculate_score(solutions_rounded)
print(f"\nRounded submission score: {rounded_score:.6f}")
print(f"Original baseline score: 70.615791")
print(f"Difference: {rounded_score - 70.615791:.6f}")